In [1]:
%cd drive/My\ Drive/Real-Time-Voice-Cloning

/content/drive/My Drive/Real-Time-Voice-Cloning


In [3]:
!pip install -q -r requirements.txt
!apt-get install -qq libportaudio2

     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 686kB 46.2MB/s 
     |████████████████████████████████| 245kB 61.9MB/s 
     |████████████████████████████████| 76.6MB 40kB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 3.8MB 61.5MB/s 
     |████████████████████████████████| 512kB 54.8MB/s 
     |████████████████████████████████| 204kB 59.5MB/s 
     |████████████████████████████████| 286kB 57.8MB/s 
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1_amd64.deb ...
Unpacking libportaudio2:amd64 (19.6.0-1) ...
Setting up libportaudio2:amd64 (19.6.0-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1) ...
/sbin/ldconfig.real: /usr/local/lib/python3.

In [2]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output


class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

In [3]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("encoder/saved_models/pretrained.pt")
vocoder_weights = Path("vocoder/saved_models/pretrained/pretrained.pt")
syn_dir = Path("synthesizer/saved_models/logs-pretrained/taco_pretrained")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

/content/drive/My Drive/Real-Time-Voice-Cloning/encoder/audio.py:13: UserWarning: Unable to import 'webrtcvad'. This package enables noise removal and is recommended.
  warn("Unable to import 'webrtcvad'. This package enables noise removal and is recommended.")


Loaded encoder "pretrained.pt" trained to step 1564501
Found synthesizer "pretrained" trained to step 278000
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


In [4]:
#@title Deep vocoder
def synth():
  text = "Hi Edouard, Happy Birthday! There is a song I sang for you." #@param {type:"string"}
  print("Now recording for 10 seconds, say what you will...")
  record(10)
  print("Audio recording complete")
  in_fpath = Path("audio.wav")
  reprocessed_wav = encoder.preprocess_wav(in_fpath)
  original_wav, sampling_rate = librosa.load(in_fpath)
  preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
  embed = encoder.embed_utterance(preprocessed_wav)
  print("Synthesizing new audio...")
  with io.capture_output() as captured:
    specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  display(Audio(generated_wav, rate=synthesizer.sample_rate))
InvokeButton('Start recording', synth)

In [24]:
text = "Victor, Happy birthday, how is it going?"
in_fpath = Path("sample.wav")
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)
print("Synthesizing new audio...")
with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))

Synthesizing new audio...
{| ████████████████ 66500/67200 | Batch Size: 7 | Gen Rate: 9.7kHz | }

In [30]:
import wave, struct
def save_wave_file(filename, data, rate):
    f = wave.open(filename, 'wb')
    nchannels = 1
    sampwidth = 2
    framerate = rate
    nframes = len(data)
    comptype = "NONE"
    compname = "not compressed"

    f.setparams((nchannels, sampwidth, framerate, nframes, comptype, compname))

    for s in data:
        # write the audio frames to file
        f.writeframes(struct.pack('h', int(s)))

In [33]:
save_wave_file('output.wav', data=generated_wav*20000, rate=synthesizer.sample_rate)

In [ ]:
with open('audio.wav','wb+') as f:
    f.write(b)

In [47]:
text = "Hi Victor, happy birthday Victor, happy birthday"
in_fpath = Path("audio.wav")
reprocessed_wav = encoder.preprocess_wav(in_fpath)
original_wav, sampling_rate = librosa.load(in_fpath)
preprocessed_wav = encoder.preprocess_wav(original_wav, sampling_rate)
embed = encoder.embed_utterance(preprocessed_wav)
print("Synthesizing new audio...")
with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
generated_wav = vocoder.infer_waveform(specs[0])
generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
display(Audio(generated_wav, rate=synthesizer.sample_rate))

Synthesizing new audio...
{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 6.6kHz | }

In [20]:
original_wav[1000:]

array([ 0.0010409 ,  0.00092999,  0.00086668, ..., -0.00059581,
       -0.00023186, -0.00015655], dtype=float32)